# Example experiment

Simulate control system of two robots trying to perform a rendevous. The control system is a second order system with damping. The governing differential equations are shown below:

### Solve coupled DE's for rendevous control system

$$\frac{d^2 \textbf{r}_1}{dt^2} = \nu_1 \frac{d \textbf{r}_1}{dt} + K_1 (\textbf{r}_2 - \textbf{r}_1)$$

$$\frac{d^2 \textbf{r}_2}{dt^2} = \nu_2 \frac{d \textbf{r}_2}{dt} + K_2 (\textbf{r}_1 - \textbf{r}_2)$$

Write as first order odes

$$\frac{d \textbf{v}_1}{dt} = \nu_1 \textbf{v}_1 + K_1 (\textbf{r}_2 - \textbf{r}_1)$$

$$\frac{d \textbf{v}_2}{dt} = \nu_2 \textbf{v}_2 + K_2 (\textbf{r}_1 - \textbf{r}_2)$$

$$\frac{d \textbf{r}_1}{dt} = \textbf{v}_1$$

$$\frac{d \textbf{r}_2}{dt} = \textbf{v}_2$$

In [1]:
import writefile_run
filename = 'rendevouz_control.py'

In [2]:
%%writefile_run $filename

# check if in an interactive environment
IPY = True
try:
    get_ipython()
except:
    IPY = False

In [3]:
%%writefile_run $filename -a


from sacred import Experiment
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import ode
from sacred.observers import FileStorageObserver
from visdom_observer import VisdomObserver
import time
import os

In [4]:
import visdom_observer

In [5]:
plt.style.use(['dark_background'])
plt.rcParams['axes.grid']=True
plt.rcParams['font.size']=15
plt.rcParams['figure.figsize']=18,7

In [6]:
%%writefile_run $filename -a


ex = Experiment('rendevouz_control', interactive=IPY)
fobs = FileStorageObserver.create('my_runs')
ex.observers.append(fobs)
ex.observers.append(VisdomObserver())

In [7]:
%%writefile_run $filename -a


@ex.config
def system_config():
    """System parameters"""
    # 'spring' constants
    K1 = 1
    K2 = 1
    
    # damping constants
    nu1 = -1
    nu2 = -1
    dim = 2

In [8]:
%%writefile_run $filename -a


@ex.config
def solver_config():
    """Config for solver"""
    steps = 1000 # steps to solve
    dt = 0.01 # time resolution
    delay = 0.01 # added delay in loop to simulate work
    
    save_fig = True # whether to save a figure showing locations of the bodies
    
    # the configs below only matter is save_fig is True
    save_fig_steps=100 # steps between which to save figure of the bodies
    dpi=50 # dpi of saved figure

In [9]:
%%writefile_run $filename -a


@ex.config
def init_value_config():
    """Initial values"""
    v1_init = [-1,0]
    v2_init = [1,0]
    r1_init = [0,1]
    r2_init = [0,-1]

    y_init = np.array([v1_init, 
                    v2_init, 
                    r1_init, 
                    r2_init]).reshape(-1)

In [10]:
r = ex._create_run('print_config')

globals().update(r.config)

In [11]:
%%writefile_run $filename -a


@ex.capture
def f(t, y, K1=1, K2=1, nu1=-1, nu2=-1, dim=2):
    """Returns derivative of vector of vars.
    dim*4 variable vector: v1, v2, r1, r2"""
    
    v1, v2, r1, r2 = y.reshape(4,dim)
        
    v1_prime = nu1*v1 + K1*(r2-r1)
    v2_prime = nu2*v2 + K2*(r1-r2)
    r1_prime = v1
    r2_prime = v2
    
    return np.concatenate([v1_prime, 
                        v2_prime, 
                        r1_prime, 
                        r2_prime])

In [12]:
f(1,np.ones(8))

array([-1., -1., -1., -1.,  1.,  1.,  1.,  1.])

In [13]:
%%writefile_run $filename -a


@ex.capture
def run_expt(y_init, dt, steps, dim, delay, save_fig, save_fig_steps, dpi):
    r = ode(f).set_initial_value(y_init)

    fig, ax = plt.subplots(1,1)
    
    plt.close(fig)
    
    t = dt
    y = [y_init]
    for i in range(steps):
        step = r.integrate(t)
        y.append(step)
        t+=dt
        
        v1, v2, r1, r2 = step.reshape(4,dim)
        
        # log metrics
        ex.log_scalar('rel_distance', np.linalg.norm(r1-r2))
        ex.log_scalar('rel_velocity', np.linalg.norm(v1-v2))
        
        arry = np.array(y)
        ax.axis('off')
        ax.scatter(arry[:,dim*2::dim], arry[:,dim*2+1::dim], c=['red', 'blue'])
        
        # save figure
        if save_fig and i%save_fig_steps==0:
            d = fobs.dir or 'figures'
            path = os.path.join(d,'fig.png')
            fig.savefig(path, bbox_inches='tight',dpi=dpi)
            ex.add_artifact(path)
            
        # extra delay to simulate work
        time.sleep(delay)
        
    y = np.array(y)
    
    return y

In [14]:
r = ex._create_run('print_config')

y = run_expt()

y[:,dim*2::dim], y[:,dim*2+1::dim]

plt.scatter(y[:,dim*2::dim], y[:,dim*2+1::dim], c=['red', 'blue'])
plt.show()

In [15]:
%%writefile_run $filename -a -dr


@ex.automain
def main():
    return run_expt()

!python rendevouz_control.py with delay=0